# Here is a complete, educational Python project for Video Analytics using classical computer vision techniques (no deep learning / neural networks).

This project demonstrates several key concepts from your course topics:

Motion estimation (via dense optical flow visualization)

Background subtraction (MOG2 and KNN)

Object tracking (MeanShift + CamShift)

Simple analytics: people motion heat map, retail_store analysis


# Video Analytics – Retail Store Project

# Features:

  • Background subtraction (MOG2 + KNN)

  • Dense optical flow visualization (Farneback)

  • CamShift object tracking (manual ROI selection)

   • Basic analytics:

   - Number of moving objects per frame

   - Dwell time estimation (how long objects stay in scene)
   - Simple motion heatmap

<hr>

In [3]:
# =========================================================
# Retail Store Video Analytics System
# Customer Tracking + Heatmap + Loitering Detection
# =========================================================

# pip install ultralytics opencv-python numpy

import cv2
import numpy as np
from ultralytics import YOLO

# -----------------------------
# CONFIGURATION
# -----------------------------
VIDEO_PATH = "/retail_store.mp4"
OUTPUT_PATH = "retail_output.mp4"
MODEL_PATH = "yolov8n.pt"

PERSON_CLASS = 0  # COCO class for person

LOITER_TIME_THRESHOLD = 5      # seconds
RESIZE_WIDTH = 960
RESIZE_HEIGHT = 540

# -----------------------------
# LOAD MODEL
# -----------------------------
model = YOLO(MODEL_PATH)

# -----------------------------
# VIDEO SETUP
# -----------------------------
cap = cv2.VideoCapture(VIDEO_PATH)
fps = cap.get(cv2.CAP_PROP_FPS)
if fps == 0:
    fps = 30

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(
    OUTPUT_PATH,
    fourcc,
    fps,
    (RESIZE_WIDTH, RESIZE_HEIGHT)
)

# -----------------------------
# TRACKING MEMORY
# -----------------------------
paths = {}
entry_time = {}
loiter_alert = set()

heatmap = np.zeros((RESIZE_HEIGHT, RESIZE_WIDTH), dtype=np.float32)

print("Retail analytics running...")

# =========================================================
# MAIN LOOP
# =========================================================
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (RESIZE_WIDTH, RESIZE_HEIGHT))

    results = model.track(frame, persist=True, conf=0.3)

    customer_count = 0

    if results[0].boxes.id is not None:

        boxes = results[0].boxes.xyxy.cpu().numpy()
        ids = results[0].boxes.id.cpu().numpy()
        classes = results[0].boxes.cls.cpu().numpy()

        for box, track_id, cls in zip(boxes, ids, classes):

            if int(cls) != PERSON_CLASS:
                continue

            customer_count += 1

            x1, y1, x2, y2 = map(int, box)
            cx = int((x1 + x2) / 2)
            cy = int((y1 + y2) / 2)

            # -----------------------------
            # PATH TRACKING
            # -----------------------------
            if track_id not in paths:
                paths[track_id] = []
                entry_time[track_id] = 0

            paths[track_id].append((cx, cy))
            entry_time[track_id] += 1

            # -----------------------------
            # HEATMAP UPDATE
            # -----------------------------
            heatmap[cy, cx] += 1

            # -----------------------------
            # DRAW BOUNDING BOX
            # -----------------------------
            color = (0, 255, 0)

            # Loitering Detection
            time_spent = entry_time[track_id] / fps
            if time_spent > LOITER_TIME_THRESHOLD:
                color = (0, 0, 255)
                loiter_alert.add(track_id)

            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

            cv2.putText(
                frame,
                f"ID:{int(track_id)}",
                (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                color,
                2
            )

            cv2.putText(
                frame,
                f"{int(time_spent)}s",
                (x1, y2 + 15),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                color,
                2
            )

            # Draw movement trail
            for i in range(1, len(paths[track_id])):
                cv2.line(frame,
                         paths[track_id][i-1],
                         paths[track_id][i],
                         (255, 0, 0), 2)

    # -----------------------------
    # DISPLAY CUSTOMER COUNT
    # -----------------------------
    cv2.putText(frame,
                f"Customers: {customer_count}",
                (50, 50),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (0, 255, 255),
                2)

    # -----------------------------
    # HEATMAP OVERLAY
    # -----------------------------
    heatmap_blur = cv2.GaussianBlur(heatmap, (15, 15), 0)
    heatmap_norm = cv2.normalize(heatmap_blur, None, 0, 255, cv2.NORM_MINMAX)
    heatmap_color = cv2.applyColorMap(
        heatmap_norm.astype(np.uint8),
        cv2.COLORMAP_JET
    )

    overlay = cv2.addWeighted(frame, 0.7, heatmap_color, 0.3, 0)

    out.write(overlay)

# =========================================================
# CLEANUP
# =========================================================
cap.release()
out.release()
cv2.destroyAllWindows()

print("Retail analytics completed ✅")
print("Output saved as:", OUTPUT_PATH)


Retail analytics running...
requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
Using Python 3.12.12 environment at: /usr
Resolved 2 packages in 175ms
Prepared 1 package in 51ms
Installed 1 package in 2ms
 + lap==0.5.12

requirements: AutoUpdate success ✅ 0.6s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


0: 384x640 12 persons, 1 backpack, 84.8ms
Speed: 8.9ms preprocess, 84.8ms inference, 40.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 persons, 1 backpack, 8.9ms
Speed: 2.6ms preprocess, 8.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 persons, 2 backpacks, 10.1ms
Speed: 2.8ms preprocess, 10.1ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 persons, 2 backpacks, 9.8ms
Speed: 2.7ms preprocess, 9.8ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 persons, 2 backpacks, 9.7ms
Speed: 2.8ms 